In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from pytorch_metric_learning.losses import SupConLoss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

NUM_CLASSES = 47   
class EMGDataset(Dataset):
    def __init__(self, x_path, y_path):
        self.X = np.load(x_path)
        self.y = np.argmax(np.load(y_path), axis=1)  

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return (
            torch.tensor(self.X[idx], dtype=torch.float32),
            torch.tensor(self.y[idx], dtype=torch.long)
        )

class CNNEncoder(nn.Module):
    def __init__(self, emb_dim=256):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv1d(6, 64, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(64, 64, kernel_size=10),
            nn.ReLU(),
            nn.MaxPool1d(3),

            nn.Conv1d(64, 256, kernel_size=10),
            nn.ReLU(),
            nn.Conv1d(256, 256, kernel_size=10),
            nn.ReLU(),

            nn.AdaptiveAvgPool1d(1)
        )

        self.fc = nn.Linear(256, emb_dim)

    def forward(self, x):
        x = x.permute(0, 2, 1)  
        x = self.features(x).squeeze(-1)
        z = self.fc(x)
        return z
    
class Classifier(nn.Module):
    def __init__(self, emb_dim=256, num_classes=NUM_CLASSES):
        super().__init__()
        self.fc = nn.Linear(emb_dim, num_classes)

    def forward(self, z):
        return self.fc(z)

class FullModel(nn.Module):
    def __init__(self, emb_dim=256, num_classes=NUM_CLASSES):
        super().__init__()
        self.encoder = CNNEncoder(emb_dim)
        self.classifier = Classifier(emb_dim, num_classes)

    def forward(self, x):
        z = self.encoder(x)
        logits = self.classifier(z)
        return z, logits

class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.best_acc = -1
        self.counter = 0
        self.best_state = None

    def step(self, acc, model):
        if acc > self.best_acc:
            self.best_acc = acc
            self.best_state = {k: v.cpu() for k, v in model.state_dict().items()}
            self.counter = 0
            return False
        else:
            self.counter += 1
            return self.counter >= self.patience

    def restore(self, model):
        model.load_state_dict(self.best_state)

def train_and_eval(
    model,
    train_loader,
    test_loader,
    supcon_loss,
    alpha=0.5,
    epochs=500,
    patience=10
):
    ce_loss = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=5e-4)
    stopper = EarlyStopping(patience)

    for epoch in range(epochs):
        model.train()
        for x, y in train_loader:
            x, y = x.to(device), y.to(device)

            z, logits = model(x)

            z_norm = F.normalize(z, dim=1)  
            loss = ce_loss(logits, y) + alpha * supcon_loss(z_norm, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        correct, total = 0, 0
        with torch.no_grad():
            for x, y in test_loader:
                x, y = x.to(device), y.to(device)
                _, logits = model(x)
                pred = logits.argmax(dim=1)
                correct += (pred == y).sum().item()
                total += y.size(0)

        acc = correct / total
        print(f"Epoch {epoch+1:02d} | Val Acc: {acc:.4f}")

        if stopper.step(acc, model):
            print("Early stopping triggered")
            break

    stopper.restore(model)
    return stopper.best_acc

BASE = "models/Data/Data/47_classes/UserDependenet"  
supcon_loss = SupConLoss(temperature=0.07)

all_accs = []

for split in range(1, 11):
    print(f"\n===== SPLIT {split} =====")

    train_ds = EMGDataset(
        f"{BASE}/Train/X_train_{split}.npy",
        f"{BASE}/Train/y_train_{split}.npy"
    )
    test_ds = EMGDataset(
        f"{BASE}/Test/X_test_{split}.npy",
        f"{BASE}/Test/y_test_{split}.npy"
    )

    train_loader = DataLoader(train_ds, batch_size=128, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=128, shuffle=False)

    model = FullModel(
        emb_dim=256,
        num_classes=NUM_CLASSES
    ).to(device)

    acc = train_and_eval(
        model,
        train_loader,
        test_loader,
        supcon_loss,
        alpha=0.5,
        epochs=500,
        patience=10
    )

    print(f"Split {split} Accuracy: {acc:.4f}")
    all_accs.append(acc)

print("\nFINAL AVERAGE ACCURACY:", np.mean(all_accs))

Device: cuda

===== SPLIT 1 =====
Epoch 01 | Val Acc: 0.0213
Epoch 02 | Val Acc: 0.1298
Epoch 03 | Val Acc: 0.4213
Epoch 04 | Val Acc: 0.5128
Epoch 05 | Val Acc: 0.5936
Epoch 06 | Val Acc: 0.6489
Epoch 07 | Val Acc: 0.6819
Epoch 08 | Val Acc: 0.7606
Epoch 09 | Val Acc: 0.7340
Epoch 10 | Val Acc: 0.7266
Epoch 11 | Val Acc: 0.7989
Epoch 12 | Val Acc: 0.8160
Epoch 13 | Val Acc: 0.8053
Epoch 14 | Val Acc: 0.8074
Epoch 15 | Val Acc: 0.8362
Epoch 16 | Val Acc: 0.7979
Epoch 17 | Val Acc: 0.8372
Epoch 18 | Val Acc: 0.8181
Epoch 19 | Val Acc: 0.8287
Epoch 20 | Val Acc: 0.8128
Epoch 21 | Val Acc: 0.8202
Epoch 22 | Val Acc: 0.8330
Epoch 23 | Val Acc: 0.8426
Epoch 24 | Val Acc: 0.8372
Epoch 25 | Val Acc: 0.8404
Epoch 26 | Val Acc: 0.8394
Epoch 27 | Val Acc: 0.8372
Epoch 28 | Val Acc: 0.8638
Epoch 29 | Val Acc: 0.8426
Epoch 30 | Val Acc: 0.8436
Epoch 31 | Val Acc: 0.8340
Epoch 32 | Val Acc: 0.8426
Epoch 33 | Val Acc: 0.8564
Epoch 34 | Val Acc: 0.8564
Epoch 35 | Val Acc: 0.8436
Epoch 36 | Val Acc: 0